In [ ]:
!pip install -q datasets
!pip install rouge

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
import json
from typing import List, Dict, Any
import random
from collections import Counter

class PromptTester:
    def __init__(self, model_name: str, hf_token: str):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            token=hf_token,
            device_map="auto",
            torch_dtype=torch.bfloat16
        )
        self.dataset = load_dataset("n3Er/FinQA-Infix")
        self.examples = self.get_curated_examples()

    def categorize_operation(self, operation: str) -> str:
        """Categorize the type of operation"""
        if any(op in operation for op in ["table_max", "table_min", "table_sum", "table_average"]):
            return "table"
        elif any(op in operation for op in [">", "<", "=="]):
            return "boolean"
        elif "^" in operation:
            return "exponential"
        else:
            return "arithmetic"

    def get_curated_examples(self) -> List[Dict]:
        """Get diverse examples covering all operation types and operators"""
        train_data = list(self.dataset['train'])

        # Filter for examples with valid operations and answers
        valid_examples = [
            ex for ex in train_data
            if ex['expanded_program_re'] and ex['answer'] is not None
        ]

        # Define operation subtypes we want to include
        operation_subtypes = {
            "arithmetic": {
                "addition": "+",
                "subtraction": "-",
                "multiplication": "*",
                "division": "/"
            },
            "table": {
                "sum": "table_sum",
                "max": "table_max",
                "min": "table_min",
                "average": "table_average"
            },
            "boolean": {
                "greater": ">",
                "less": "<",
                "equal": "=="
            },
            "exponential": {
                "power": "^"
            }
        }

        # Initialize collection of examples
        selected_examples = []

        # For each main operation type
        for op_type, subtypes in operation_subtypes.items():
            for subtype, operator in subtypes.items():
                # Find examples containing this operator
                matching_examples = [
                    ex for ex in valid_examples
                    if operator in ex['expanded_program_re']
                    and not any(operator in selected['expanded_program_re'] for selected in selected_examples)
                ]

                # Select the first matching example
                for example in matching_examples:
                    if len(selected_examples) < 8:  # Limit to 8 total examples
                        selected_examples.append(example)
                        break

        return selected_examples

    def format_table(self, table: List[List[Any]]) -> str:
        if not table:
            return ""
        return "\n".join(" | ".join(str(cell) for cell in row) for row in table)

    def format_context(self, example: Dict) -> str:
        """Format context consistently"""
        parts = []
        if example['pre_text']:
            parts.append("Context before:")
            parts.append(' '.join(example['pre_text']))
        if example['table']:
            parts.append("\nTable:")
            parts.append(self.format_table(example['table']))
        if example['post_text']:
            parts.append("\nContext after:")
            parts.append(' '.join(example['post_text']))
        return "\n".join(parts)

    def create_prompt(self, example: Dict) -> str:
        """Create prompt with chain of thought reasoning using all examples"""
        demonstrations = []
        for demo in self.examples:  # Using all examples
            demo_text = f"""Question: {demo['question']}
Context:
{self.format_context(demo)}

Think through the solution:
1. Looking at the question, we need to find specific values from the context
2. The question is asking about {demo['question'].lower()}
3. To solve this, we'll need to use these values in a calculation

Operations: {demo['expanded_program_re']}
Result: {demo['answer']}
---"""
            demonstrations.append(demo_text)

        context = self.format_context(example)
        return f"""Solve each financial question by thinking through the steps carefully.

Examples:
{chr(10).join(demonstrations)}

For this question:
Question: {example['question']}
Context:
{context}

Think through your solution:
1. What specific values do you need from the context?
2. How will you use these values to get the answer?
3. What mathematical operation will give the correct result?

Now provide the operation and result:
Operations: [operations]
Result: [result]"""

    def extract_response(self, response: str) -> Dict[str, str]:
        """Extract operations and result from response"""
        operations = None
        answer = None

        lines = [line.strip() for line in response.split('\n') if line.strip()]

        for line in lines:
            if line.startswith("Operations:"):
                operations = line.replace("Operations:", "").strip()
                operations = operations.replace('[operations]', '').strip()
            elif line.startswith("Result:"):
                answer = line.replace("Result:", "").strip()
                answer = answer.replace('[result]', '').strip()

        return {
            "operations": operations,
            "answer": answer
        }

    def test_prompts(self, num_samples: int = 5) -> None:
        """Test with chain of thought prompting"""
        test_samples = []
        example_ids = {ex['id'] for ex in self.examples}
        for sample in random.sample(list(self.dataset['train']), num_samples + len(example_ids)):
            if sample['id'] not in example_ids and len(test_samples) < num_samples:
                test_samples.append(sample)

        print("\nExample Operations Used in Prompt:")
        print("=" * 50)
        for ex in self.examples:
            print(f"\nOperation Type: {self.categorize_operation(ex['expanded_program_re'])}")
            print(f"Operations: {ex['expanded_program_re']}")

        print("\nTesting with chain of thought reasoning:")
        print("=" * 50)
        for idx, sample in enumerate(test_samples, 1):
            print(f"\nTest Sample {idx}")
            print("-" * 30)
            print(f"Operation Type: {self.categorize_operation(sample['expanded_program_re'])}")
            print(f"Question: {sample['question']}")
            print(f"Expected Operations: {sample['expanded_program_re']}")
            print(f"Expected Answer: {sample['answer']}")

            # Get response with chain of thought
            prompt = self.create_prompt(sample)
            inputs = self.tokenizer(prompt, return_tensors="pt").to("cuda")
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=200,
                    temperature=0.7,
                    top_p=0.95,
                    do_sample=True,
                    num_return_sequences=1,
                    pad_token_id=self.tokenizer.eos_token_id
                )
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            extracted = self.extract_response(response)

            print("\nModel Response:")
            print(f"Operations needed: {extracted['operations']}")
            print(f"Answer: {extracted['answer']}")

if __name__ == "__main__":
    tester = PromptTester(
        model_name="Qwen/Qwen2.5-7B-Instruct",
        hf_token="your_private_token"
    )
    tester.test_prompts(num_samples=5)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6251 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/883 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1147 [00:00<?, ? examples/s]


Example Operations Used in Prompt:

Operation Type: arithmetic
Operations: (75.0 + 72.7) / 20

Operation Type: arithmetic
Operations: (959.2 - 991.1) / 991.1

Operation Type: boolean
Operations: ((607 * 18.13) * const_1000) > (3.3 * const_1000000)

Operation Type: table
Operations: table_sum(cash flows on retained interests)

Operation Type: table
Operations: table_max(cumulative foreign currency translation)

Operation Type: table
Operations: (table_max(expected volatility)) - (table_min(expected volatility))

Operation Type: table
Operations: table_average(2016)

Operation Type: exponential
Operations: 397 * ((const_1 + 2.0%)^7)

Testing with chain of thought reasoning:

Test Sample 1
------------------------------
Operation Type: arithmetic
Question: when pnc redeemed all shares of the series m preferred stock from the trust on december 10 , 2012 , what was the total cash cost of the redemption?
Expected Operations: 5001 * const_100000
Expected Answer: 500100000

Model Response:
Op

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
import json
from typing import List, Dict, Any
import random
from collections import Counter

class PromptTester:
    def __init__(self, model_name: str, hf_token: str):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            token=hf_token,
            device_map="auto",
            torch_dtype=torch.bfloat16
        )
        self.dataset = load_dataset("n3Er/FinQA-Infix")
        self.examples = self.get_curated_examples()

    def categorize_operation(self, operation: str) -> str:
        """Categorize the type of operation"""
        if any(op in operation for op in ["table_max", "table_min", "table_sum", "table_average"]):
            return "table"
        elif any(op in operation for op in [">", "<", "=="]):
            return "boolean"
        elif "^" in operation:
            return "exponential"
        else:
            return "arithmetic"

    def get_curated_examples(self) -> List[Dict]:
        """Get diverse examples covering all operation types and operators"""
        train_data = list(self.dataset['train'])

        # Filter for examples with valid operations and answers
        valid_examples = [
            ex for ex in train_data
            if ex['expanded_program_re'] and ex['answer'] is not None
        ]

        # Define operation subtypes we want to include
        operation_subtypes = {
            "arithmetic": {
                "addition": "+",
                "subtraction": "-",
                "multiplication": "*",
                "division": "/"
            },
            "table": {
                "sum": "table_sum",
                "max": "table_max",
                "min": "table_min",
                "average": "table_average"
            },
            "boolean": {
                "greater": ">",
                "less": "<",
                "equal": "=="
            },
            "exponential": {
                "power": "^"
            }
        }

        # Initialize collection of examples
        selected_examples = []

        # For each main operation type
        for op_type, subtypes in operation_subtypes.items():
            for subtype, operator in subtypes.items():
                # Find examples containing this operator
                matching_examples = [
                    ex for ex in valid_examples
                    if operator in ex['expanded_program_re']
                    and not any(operator in selected['expanded_program_re'] for selected in selected_examples)
                ]

                # Select the first matching example
                for example in matching_examples:
                    if len(selected_examples) < 8:  # Limit to 8 total examples
                        selected_examples.append(example)
                        break

        return selected_examples

    def format_table(self, table: List[List[Any]]) -> str:
        if not table:
            return ""
        return "\n".join(" | ".join(str(cell) for cell in row) for row in table)

    def format_context(self, example: Dict) -> str:
        """Format context consistently"""
        parts = []
        if example['pre_text']:
            parts.append("Context before:")
            parts.append(' '.join(example['pre_text']))
        if example['table']:
            parts.append("\nTable:")
            parts.append(self.format_table(example['table']))
        if example['post_text']:
            parts.append("\nContext after:")
            parts.append(' '.join(example['post_text']))
        return "\n".join(parts)

    def create_prompt(self, example: Dict) -> str:
        """Create prompt with chain of thought reasoning using all examples"""
        demonstrations = []
        for demo in self.examples:  # Using all examples
            demo_text = f"""Question: {demo['question']}
Context:
{self.format_context(demo)}

Think through the solution:
1. Looking at the question, we need to find specific values from the context
2. The question is asking about {demo['question'].lower()}
3. To solve this, we need to calculate:
Operations: {demo['expanded_program_re']}

4. This calculation gives us:
Result: {demo['answer']}
---"""
            demonstrations.append(demo_text)

        context = self.format_context(example)
        return f"""Solve each financial question by thinking through the steps carefully. Always show both the operations needed and the final result.

Examples:
{chr(10).join(demonstrations)}

For this question:
Question: {example['question']}
Context:
{context}

Think through your solution:
1. What specific values do you need from the context?
2. How will you use these values?
3. Write out the exact mathematical operation you'll use (using operators like +, -, *, /, table_sum, etc.)

You must provide both:
Operations: [write the exact mathematical operation]
Result: [result]"""

    def extract_response(self, response: str) -> Dict[str, str]:
        """Extract operations and result from response with better handling of formatting"""
        operations = None
        answer = None

        # Clean up the response
        lines = [line.strip() for line in response.split('\n') if line.strip()]

        # Handle variations in format
        for i, line in enumerate(lines):
            if line.startswith("Operations:") or "Operations:" in line:
                operations = line.split("Operations:")[-1].strip()
                operations = operations.replace('[write the exact mathematical operation]', '').strip()
                operations = operations.replace('[operations]', '').strip()

            elif line.startswith("Result:") or "Result:" in line:
                answer = line.split("Result:")[-1].strip()
                answer = answer.replace('[numerical answer]', '').strip()
                answer = answer.replace('[result]', '').strip()
                # Clean up common artifacts
                answer = answer.replace('---', '').strip()

        return {
            "operations": operations if operations and operations != '---' else None,
            "answer": answer if answer and answer != '---' else None
        }

    def test_prompts(self, num_samples: int = 5) -> None:
        """Test with chain of thought prompting"""
        test_samples = []
        example_ids = {ex['id'] for ex in self.examples}
        for sample in random.sample(list(self.dataset['train']), num_samples + len(example_ids)):
            if sample['id'] not in example_ids and len(test_samples) < num_samples:
                test_samples.append(sample)

        print("\nExample Operations Used in Prompt:")
        print("=" * 50)
        for ex in self.examples:
            print(f"\nOperation Type: {self.categorize_operation(ex['expanded_program_re'])}")
            print(f"Operations: {ex['expanded_program_re']}")

        print("\nTesting with chain of thought reasoning:")
        print("=" * 50)
        for idx, sample in enumerate(test_samples, 1):
            print(f"\nTest Sample {idx}")
            print("-" * 30)
            print(f"Operation Type: {self.categorize_operation(sample['expanded_program_re'])}")
            print(f"Question: {sample['question']}")
            print(f"Expected Operations: {sample['expanded_program_re']}")
            print(f"Expected Answer: {sample['answer']}")

            # Get response with chain of thought
            prompt = self.create_prompt(sample)
            inputs = self.tokenizer(prompt, return_tensors="pt").to("cuda")
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=200,
                    temperature=0.7,
                    top_p=0.95,
                    do_sample=True,
                    num_return_sequences=1,
                    pad_token_id=self.tokenizer.eos_token_id
                )
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            extracted = self.extract_response(response)

            print("\nModel Response:")
            print(f"Operations needed: {extracted['operations']}")
            print(f"Answer: {extracted['answer']}")

if __name__ == "__main__":
    tester = PromptTester(
        model_name="Qwen/Qwen2.5-7B-Instruct",
        hf_token="your_private_token"
    )
    tester.test_prompts(num_samples=10)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6251 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/883 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1147 [00:00<?, ? examples/s]


Example Operations Used in Prompt:

Operation Type: arithmetic
Operations: (75.0 + 72.7) / 20

Operation Type: arithmetic
Operations: (959.2 - 991.1) / 991.1

Operation Type: boolean
Operations: ((607 * 18.13) * const_1000) > (3.3 * const_1000000)

Operation Type: table
Operations: table_sum(cash flows on retained interests)

Operation Type: table
Operations: table_max(cumulative foreign currency translation)

Operation Type: table
Operations: (table_max(expected volatility)) - (table_min(expected volatility))

Operation Type: table
Operations: table_average(2016)

Operation Type: exponential
Operations: 397 * ((const_1 + 2.0%)^7)

Testing with chain of thought reasoning:

Test Sample 1
------------------------------
Operation Type: arithmetic
Question: what percent did net sales increase between 2018 and 2019?
Expected Operations: 18289.0 - 16285.1
Expected Answer: 12.31%

Model Response:
Operations needed: (18289.0 - 16285.1) / 16285.1 * 100
Answer: 12.3%

Test Sample 2
------------

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
import json
from typing import List, Dict, Any
import random
from collections import Counter

class PromptTester:
    def __init__(self, model_name: str, hf_token: str):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            token=hf_token,
            device_map="auto",
            torch_dtype=torch.bfloat16
        )
        self.dataset = load_dataset("n3Er/FinQA-Infix")
        self.examples = self.get_curated_examples()

    def categorize_operation(self, operation: str) -> str:
        """Categorize the type of operation"""
        if any(op in operation for op in ["table_max", "table_min", "table_sum", "table_average"]):
            return "table"
        elif any(op in operation for op in [">", "<", "=="]):
            return "boolean"
        elif "^" in operation:
            return "exponential"
        else:
            return "arithmetic"

    def get_curated_examples(self) -> List[Dict]:
        """Get diverse examples covering all operation types and operators"""
        train_data = list(self.dataset['train'])

        # Filter for examples with valid operations and answers
        valid_examples = [
            ex for ex in train_data
            if ex['expanded_program_re'] and ex['answer'] is not None
        ]

        # Define operation subtypes we want to include
        operation_subtypes = {
            "arithmetic": {
                "addition": "+",
                "subtraction": "-",
                "multiplication": "*",
                "division": "/",
                "percentage": "%"
            },
            "table": {
                "sum": "table_sum",
                "max": "table_max",
                "min": "table_min",
                "average": "table_average"
            },
            "boolean": {
                "greater": ">",
                "less": "<",
                "equal": "=="
            },
            "exponential": {
                "power": "^"
            }
        }

        # Initialize collection of examples
        selected_examples = []

        # For each main operation type
        for op_type, subtypes in operation_subtypes.items():
            for subtype, operator in subtypes.items():
                # Find examples containing this operator
                matching_examples = [
                    ex for ex in valid_examples
                    if operator in ex['expanded_program_re']
                    and not any(operator in selected['expanded_program_re'] for selected in selected_examples)
                ]

                # Select the first matching example
                for example in matching_examples:
                    if len(selected_examples) < 8:  # Limit to 8 total examples
                        selected_examples.append(example)
                        break

        return selected_examples

    def format_table(self, table: List[List[Any]]) -> str:
        if not table:
            return ""
        return "\n".join(" | ".join(str(cell) for cell in row) for row in table)

    def format_context(self, example: Dict) -> str:
        """Format context consistently"""
        parts = []
        if example['pre_text']:
            parts.append("Context before:")
            parts.append(' '.join(example['pre_text']))
        if example['table']:
            parts.append("\nTable:")
            parts.append(self.format_table(example['table']))
        if example['post_text']:
            parts.append("\nContext after:")
            parts.append(' '.join(example['post_text']))
        return "\n".join(parts)

    def create_prompt(self, example: Dict) -> str:
        """Create prompt with chain of thought reasoning and explicit calculation guidance"""
        demonstrations = []
        for demo in self.examples:  # Using all examples
            demo_text = f"""Question: {demo['question']}
Context:
{self.format_context(demo)}

Think through the solution:
1. Looking at the values in the context:
   - What specific numbers are we comparing or calculating with?
   - Is this a percentage change, ratio, or absolute difference?
2. For percentage changes: use (new_value - old_value) / old_value * 100
   For ratios: use numerator / denominator
   For differences: use new_value - old_value
3. The exact calculation needed is:
Operations: {demo['expanded_program_re']}

4. This gives us:
Result: {demo['answer']}
---"""
            demonstrations.append(demo_text)

        context = self.format_context(example)
        return f"""Solve each financial question by thinking through the steps carefully.
Always write the exact mathematical expression using actual numbers and operators (+, -, *, /, >, <, table_sum, table_max, table_average, ^, etc.).

Examples:
{chr(10).join(demonstrations)}

For this question:
Question: {example['question']}
Context:
{context}

Think through your solution:
1. Identify the exact numbers needed from the context
2. For percentage changes: use (new_value - old_value) / old_value * 100
   For ratios: use numerator / denominator
   For differences: use new_value - old_value
3. Write the complete mathematical expression

You must provide both:
Operations: [write the exact mathematical expression using numbers]
Result: [result]"""

    def extract_response(self, response: str) -> Dict[str, str]:
        """Extract operations and result with better handling of variations"""
        operations = None
        answer = None

        # Clean up the response
        lines = [line.strip() for line in response.split('\n') if line.strip()]

        # Handle variations in format
        for line in lines:
            if "Operations:" in line:
                operations = line.split("Operations:")[-1].strip()
                operations = operations.replace('[write the exact mathematical expression using numbers]', '').strip()
                operations = operations.replace('[operations]', '').strip()
                if operations == 'None' or operations == '---':
                    operations = None

            elif "Result:" in line:
                answer = line.split("Result:")[-1].strip()
                answer = answer.replace('[result]', '').strip()
                if answer == 'None' or answer == '---' or answer == '(as a number)':
                    answer = None

        return {
            "operations": operations,
            "answer": answer
        }

    def test_prompts(self, num_samples: int = 10) -> None:
        """Test with chain of thought prompting"""
        test_samples = []
        example_ids = {ex['id'] for ex in self.examples}
        for sample in random.sample(list(self.dataset['train']), num_samples + len(example_ids)):
            if sample['id'] not in example_ids and len(test_samples) < num_samples:
                test_samples.append(sample)

        print("\nExample Operations Used in Prompt:")
        print("=" * 50)
        for ex in self.examples:
            print(f"\nOperation Type: {self.categorize_operation(ex['expanded_program_re'])}")
            print(f"Operations: {ex['expanded_program_re']}")

        print("\nTesting with chain of thought reasoning:")
        print("=" * 50)
        for idx, sample in enumerate(test_samples, 1):
            print(f"\nTest Sample {idx}")
            print("-" * 30)
            print(f"Operation Type: {self.categorize_operation(sample['expanded_program_re'])}")
            print(f"Question: {sample['question']}")
            print(f"Expected Operations: {sample['expanded_program_re']}")
            print(f"Expected Answer: {sample['answer']}")

            # Get response with chain of thought
            prompt = self.create_prompt(sample)
            inputs = self.tokenizer(prompt, return_tensors="pt").to("cuda")
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=200,
                    temperature=0.7,
                    top_p=0.95,
                    do_sample=True,
                    num_return_sequences=1,
                    pad_token_id=self.tokenizer.eos_token_id
                )
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            extracted = self.extract_response(response)

            print("\nModel Response:")
            print(f"Operations needed: {extracted['operations']}")
            print(f"Answer: {extracted['answer']}")

if __name__ == "__main__":
    tester = PromptTester(
        model_name="Qwen/Qwen2.5-7B-Instruct",
        hf_token="your_private_token"
    )
    tester.test_prompts(num_samples=10)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6251 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/883 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1147 [00:00<?, ? examples/s]


Example Operations Used in Prompt:

Operation Type: arithmetic
Operations: (75.0 + 72.7) / 20

Operation Type: arithmetic
Operations: (959.2 - 991.1) / 991.1

Operation Type: boolean
Operations: ((607 * 18.13) * const_1000) > (3.3 * const_1000000)

Operation Type: arithmetic
Operations: 9896 / 23.6%

Operation Type: table
Operations: table_sum(cash flows on retained interests)

Operation Type: table
Operations: table_max(cumulative foreign currency translation)

Operation Type: table
Operations: (table_max(expected volatility)) - (table_min(expected volatility))

Operation Type: table
Operations: table_average(2016)

Testing with chain of thought reasoning:

Test Sample 1
------------------------------
Operation Type: arithmetic
Question: what portion of the total outstanding long-term debt is included in the current liabilities section as of december 31 , 2018?
Expected Operations: 124176 / 5167643
Expected Answer: 2.4%

Model Response:
Operations needed: 
Answer: 2.41

Test Sample 2

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
import json
from typing import List, Dict, Any
import random
from collections import Counter

class PromptTester:
    def __init__(self, model_name: str, hf_token: str):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            token=hf_token,
            device_map="auto",
            torch_dtype=torch.bfloat16
        )
        self.dataset = load_dataset("n3Er/FinQA-Infix")
        self.examples = self.get_curated_examples()

    def categorize_operation(self, operation: str) -> str:
        """Categorize the type of operation"""
        if any(op in operation for op in ["table_max", "table_min", "table_sum", "table_average"]):
            return "table"
        elif any(op in operation for op in [">", "<", "=="]):
            return "boolean"
        elif "^" in operation:
            return "exponential"
        else:
            return "arithmetic"

    def get_curated_examples(self) -> List[Dict]:
        """Get diverse examples covering all operation types and operators"""
        train_data = list(self.dataset['train'])

        # Filter for examples with valid operations and answers
        valid_examples = [
            ex for ex in train_data
            if ex['expanded_program_re'] and ex['answer'] is not None
        ]

        # Define operation subtypes we want to include
        operation_subtypes = {
            "arithmetic": {
                "addition": "+",
                "subtraction": "-",
                "multiplication": "*",
                "division": "/",
                "percentage": "%"
            },
            "table": {
                "sum": "table_sum",
                "max": "table_max",
                "min": "table_min",
                "average": "table_average"
            },
            "boolean": {
                "greater": ">",
                "less": "<",
                "equal": "=="
            },
            "exponential": {
                "power": "^"
            }
        }

        # Initialize collection of examples
        selected_examples = []

        # For each main operation type
        for op_type, subtypes in operation_subtypes.items():
            for subtype, operator in subtypes.items():
                # Find examples containing this operator
                matching_examples = [
                    ex for ex in valid_examples
                    if operator in ex['expanded_program_re']
                    and not any(operator in selected['expanded_program_re'] for selected in selected_examples)
                ]

                # Select the first matching example
                for example in matching_examples:
                    if len(selected_examples) < 8:  # Limit to 8 total examples
                        selected_examples.append(example)
                        break

        return selected_examples

    def format_table(self, table: List[List[Any]]) -> str:
        if not table:
            return ""
        return "\n".join(" | ".join(str(cell) for cell in row) for row in table)

    def format_context(self, example: Dict) -> str:
        """Format context consistently"""
        parts = []
        if example['pre_text']:
            parts.append("Context before:")
            parts.append(' '.join(example['pre_text']))
        if example['table']:
            parts.append("\nTable:")
            parts.append(self.format_table(example['table']))
        if example['post_text']:
            parts.append("\nContext after:")
            parts.append(' '.join(example['post_text']))
        return "\n".join(parts)

    def create_prompt(self, example: Dict) -> str:
        """Create prompt with chain of thought reasoning"""
        demonstrations = []
        for demo in self.examples:  # Using all examples
            demo_text = f"""Question: {demo['question']}
Context:
{self.format_context(demo)}
Think through the solution:
1. Looking at the values in the context:
   - What specific numbers are we comparing or calculating with?
   - Is this a percentage change, ratio, or absolute difference?
2. For percentage changes: use (new_value - old_value) / old_value * 100
   For ratios: use numerator / denominator
   For differences: use new_value - old_value
3. The exact calculation needed is:
Operations: {demo['expanded_program_re']}
4. This gives us:
Result: {demo['answer']}
---"""
            demonstrations.append(demo_text)

        context = self.format_context(example)
        return f"""Solve each financial question by thinking through the steps carefully.
Always write the exact mathematical expression using actual numbers and operators (+, -, *, /, >, <, table_sum, table_max, table_average, ^, etc.).

Examples:
{chr(10).join(demonstrations)}

For this question:
Question: {example['question']}
Context:
{context}

Think through your solution:
1. Identify the exact numbers needed from the context
2. For percentage changes: use (new_value - old_value) / old_value * 100
   For ratios: use numerator / denominator
   For differences: use new_value - old_value
3. Write the complete mathematical expression

You must provide both:
Operations: [write the exact mathematical expression using numbers]
Result: [result]"""

    def extract_response(self, response: str) -> Dict[str, str]:
        """Extract operations and result with better handling of variations"""
        operations = None
        answer = None

        # Clean up the response
        lines = [line.strip() for line in response.split('\n') if line.strip()]

        # Handle variations in format
        for line in lines:
            if "Operations:" in line:
                operations = line.split("Operations:")[-1].strip()
                operations = operations.replace('[write the exact mathematical expression using numbers]', '').strip()
                operations = operations.replace('[operations]', '').strip()
                if operations == 'None' or operations == '---':
                    operations = None

            elif "Result:" in line:
                answer = line.split("Result:")[-1].strip()
                answer = answer.replace('[result]', '').strip()
                if answer == 'None' or answer == '---' or answer == '(as a number)':
                    answer = None

        return {
            "operations": operations,
            "answer": answer
        }

    def evaluate_response(self, expected_ops: str, predicted_ops: str) -> Dict[str, float]:
        """Evaluate the response using ROUGE metrics"""
        try:
            from rouge import Rouge
        except ImportError:
            print("Rouge package not found. Installing rouge...")
            import subprocess
            import sys
            subprocess.check_call([sys.executable, "-m", "pip", "install", "rouge"])
            from rouge import Rouge

        metrics = {}

        # Initialize Rouge for operation comparison
        rouge = Rouge()

        # Evaluate operations using Rouge if both exist
        if expected_ops and predicted_ops:
            try:
                # Calculate Rouge scores
                rouge_scores = rouge.get_scores(predicted_ops, expected_ops)[0]
                metrics['rouge1_f'] = rouge_scores['rouge-1']['f']
                metrics['rouge2_f'] = rouge_scores['rouge-2']['f']
                metrics['rougeL_f'] = rouge_scores['rouge-l']['f']
            except Exception:
                metrics['rouge1_f'] = 0.0
                metrics['rouge2_f'] = 0.0
                metrics['rougeL_f'] = 0.0
        else:
            metrics['rouge1_f'] = 0.0
            metrics['rouge2_f'] = 0.0
            metrics['rougeL_f'] = 0.0

        return metrics

    def test_prompts(self, num_samples: int = 10) -> None:
        """Test with chain of thought prompting and evaluate using ROUGE"""
        test_samples = []
        example_ids = {ex['id'] for ex in self.examples}
        for sample in random.sample(list(self.dataset['train']), num_samples + len(example_ids)):
            if sample['id'] not in example_ids and len(test_samples) < num_samples:
                test_samples.append(sample)

        print("\nExample Operations Used in Prompt:")
        print("=" * 50)
        for ex in self.examples:
            print(f"\nOperation Type: {self.categorize_operation(ex['expanded_program_re'])}")
            print(f"Operations: {ex['expanded_program_re']}")

        print("\nTesting with chain of thought reasoning:")
        print("=" * 50)

        # Collect metrics across all samples
        all_metrics = []

        for idx, sample in enumerate(test_samples, 1):
            print(f"\nTest Sample {idx}")
            print("-" * 30)
            print(f"Operation Type: {self.categorize_operation(sample['expanded_program_re'])}")
            print(f"Question: {sample['question']}")
            print(f"Expected Operations: {sample['expanded_program_re']}")
            print(f"Expected Answer: {sample['answer']}")

            # Get response with chain of thought
            prompt = self.create_prompt(sample)
            inputs = self.tokenizer(prompt, return_tensors="pt").to("cuda")
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=200,
                    temperature=0.7,
                    top_p=0.95,
                    do_sample=True,
                    num_return_sequences=1,
                    pad_token_id=self.tokenizer.eos_token_id
                )
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            extracted = self.extract_response(response)

            print("\nModel Response:")
            print(f"Operations needed: {extracted['operations']}")
            print(f"Answer: {extracted['answer']}")

            # Evaluate the response using ROUGE
            metrics = self.evaluate_response(
                sample['expanded_program_re'],
                extracted['operations']
            )
            all_metrics.append(metrics)

            print("\nROUGE Scores:")
            print(f"ROUGE-1 F1: {metrics['rouge1_f']:.3f}")
            print(f"ROUGE-2 F1: {metrics['rouge2_f']:.3f}")
            print(f"ROUGE-L F1: {metrics['rougeL_f']:.3f}")

        # Calculate and display average metrics
        avg_metrics = {
            metric: sum(m[metric] for m in all_metrics) / len(all_metrics)
            for metric in all_metrics[0].keys()
        }

        print("\nOverall ROUGE Scores:")
        print("=" * 50)
        print(f"Average ROUGE-1 F1: {avg_metrics['rouge1_f']:.3f}")
        print(f"Average ROUGE-2 F1: {avg_metrics['rouge2_f']:.3f}")
        print(f"Average ROUGE-L F1: {avg_metrics['rougeL_f']:.3f}")

if __name__ == "__main__":
    tester = PromptTester(
        model_name="Qwen/Qwen2.5-7B-Instruct",
        hf_token="your_private_token"
    )
    tester.test_prompts(num_samples=10)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6251 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/883 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1147 [00:00<?, ? examples/s]


Example Operations Used in Prompt:

Operation Type: arithmetic
Operations: (75.0 + 72.7) / 20

Operation Type: arithmetic
Operations: (959.2 - 991.1) / 991.1

Operation Type: boolean
Operations: ((607 * 18.13) * const_1000) > (3.3 * const_1000000)

Operation Type: arithmetic
Operations: 9896 / 23.6%

Operation Type: table
Operations: table_sum(cash flows on retained interests)

Operation Type: table
Operations: table_max(cumulative foreign currency translation)

Operation Type: table
Operations: (table_max(expected volatility)) - (table_min(expected volatility))

Operation Type: table
Operations: table_average(2016)

Testing with chain of thought reasoning:

Test Sample 1
------------------------------
Operation Type: arithmetic
Question: what percentage of total miles were other main line in 2015?
Expected Operations: 7012 / 51439
Expected Answer: 14%

Model Response:
Operations needed: (7012 / 51439) * 100
Answer: 14 % (rounded to the nearest whole number) ---

ROUGE Scores:
ROUGE-1